# ✅ Validación de Datos con LLMs

Objetivo: usar LLMs para detectar anomalías, validar calidad de datos, clasificar errores, y generar reglas de validación de forma inteligente.

- Duración: 90 min
- Dificultad: Media/Alta
- Stack: OpenAI, Great Expectations, Pandas

## 1. Detección de anomalías semánticas

In [ ]:
import os
import pandas as pd
from openai import OpenAI

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def detect_anomaly(value: str, context: str) -> dict:
    """Detecta si un valor es anómalo en su contexto."""
    prompt = f'''
Contexto: {context}
Valor: {value}

¿Es este valor anómalo o incorrecto? Responde en JSON:
{{
  "is_anomaly": true/false,
  "confidence": 0-100,
  "reason": "explicación",
  "suggested_fix": "valor corregido o null"
}}
'''
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0
    )
    import json
    return json.loads(resp.choices[0].message.content)

# Ejemplos
casos = [
    {'valor': 'Nueva Yorkk', 'contexto': 'Columna: ciudad (ciudades de USA)'},
    {'valor': '999', 'contexto': 'Columna: edad (años de personas)'},
    {'valor': 'admin@example.com', 'contexto': 'Columna: email de clientes reales'}
]

for caso in casos:
    result = detect_anomaly(caso['valor'], caso['contexto'])
    print(f"Valor: {caso['valor']}")
    print(f"Anomalía: {result['is_anomaly']} (confianza={result['confidence']}%)")
    print(f"Razón: {result['reason']}")
    if result['suggested_fix']:
        print(f"Sugerencia: {result['suggested_fix']}")
    print()

## 2. Clasificación de errores en datos

In [ ]:
def classify_data_issue(issue_description: str) -> str:
    """Clasifica el tipo de problema de datos."""
    prompt = f'''
Clasifica este problema de datos en UNA categoría:
- DUPLICATES: registros duplicados
- NULLS: valores faltantes
- FORMAT: formato incorrecto
- OUTLIER: valores fuera de rango
- INCONSISTENCY: datos inconsistentes entre fuentes
- FRESHNESS: datos desactualizados

Problema: {issue_description}

Categoría:
'''
    resp = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[{'role':'user','content':prompt}],
        temperature=0
    )
    return resp.choices[0].message.content.strip()

issues = [
    'La tabla tiene 500 filas con cliente_id = NULL',
    'Fechas en formato DD/MM/YYYY pero esperamos YYYY-MM-DD',
    'Última actualización hace 7 días pero debe ser diaria',
    'Misma transacción aparece 3 veces con diferentes IDs'
]

for issue in issues:
    categoria = classify_data_issue(issue)
    print(f'➡️ "{issue}"')
    print(f'   Categoría: {categoria}\n')

## 3. Generación de reglas de validación

In [ ]:
def generate_validation_rules(column_name: str, sample_data: list, description: str = '') -> str:
    """Genera reglas de Great Expectations."""
    prompt = f'''
Genera expectativas de Great Expectations (Python) para validar esta columna:

Columna: {column_name}
Descripción: {description}
Muestra de datos: {sample_data}

Genera código Python con expect_* methods. Ejemplos:
- expect_column_values_to_not_be_null
- expect_column_values_to_be_between
- expect_column_values_to_match_regex

Código:
'''
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.1
    )
    return resp.choices[0].message.content.strip().replace('```python','').replace('```','')

# Ejemplo
rules = generate_validation_rules(
    column_name='email',
    sample_data=['user@example.com', 'admin@test.org', 'info@company.co'],
    description='Emails de clientes'
)

print(rules)

## 4. Validación batch con LLM

In [ ]:
def validate_batch(df: pd.DataFrame, rules: dict) -> pd.DataFrame:
    """Valida DataFrame según reglas inferidas por LLM."""
    results = []
    
    for col, rule_desc in rules.items():
        sample = df[col].dropna().head(10).tolist()
        
        prompt = f'''
Valida si estos valores cumplen la regla:
Regla: {rule_desc}
Valores: {sample}

Responde con porcentaje de conformidad (0-100) y problemas encontrados.
'''
        
        resp = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=[{'role':'user','content':prompt}],
            temperature=0
        )
        
        results.append({
            'columna': col,
            'regla': rule_desc,
            'resultado': resp.choices[0].message.content
        })
    
    return pd.DataFrame(results)

# Datos de prueba
df_test = pd.DataFrame({
    'edad': [25, 30, 200, 45, -5],
    'email': ['a@b.com', 'invalido', 'test@x.org', None, 'ok@mail.com']
})

validation_rules = {
    'edad': 'Debe estar entre 0 y 120',
    'email': 'Debe ser email válido o null'
}

validation_report = validate_batch(df_test, validation_rules)
print(validation_report)

## 5. Explicación de anomalías

In [ ]:
def explain_outlier(value: float, stats: dict) -> str:
    """Explica por qué un valor es outlier en lenguaje natural."""
    prompt = f'''
Valor: {value}
Estadísticas de la columna:
- Media: {stats['mean']}
- Desviación estándar: {stats['std']}
- Min: {stats['min']}
- Max: {stats['max']}

Explica en 2-3 frases por qué este valor es anómalo y qué puede indicar.
'''
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.3
    )
    return resp.choices[0].message.content.strip()

# Ejemplo
outlier_explanation = explain_outlier(
    value=50000,
    stats={'mean': 120, 'std': 35, 'min': 50, 'max': 250}
)

print('Explicación del outlier:')
print(outlier_explanation)

## 6. Sugerencia de limpieza de datos

In [ ]:
def suggest_data_cleaning(df_sample: pd.DataFrame) -> str:
    """Sugiere pasos de limpieza basados en muestra."""
    info = {
        'columns': df_sample.columns.tolist(),
        'dtypes': df_sample.dtypes.astype(str).to_dict(),
        'nulls': df_sample.isnull().sum().to_dict(),
        'sample': df_sample.head(3).to_dict()
    }
    
    prompt = f'''
Analiza este DataFrame y sugiere pasos de limpieza en orden de prioridad:

{info}

Lista numerada de acciones de limpieza con código Pandas cuando sea relevante.
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.2
    )
    
    return resp.choices[0].message.content

messy_df = pd.DataFrame({
    'fecha': ['2024-01-01', '01/02/2024', None, '2024-03-15'],
    'monto': ['100', '200.5', 'N/A', '300'],
    'categoria': ['  ventas', 'VENTAS', 'Ventas ', 'marketing']
})

cleaning_plan = suggest_data_cleaning(messy_df)
print(cleaning_plan)

## 7. Validación de coherencia entre tablas

In [ ]:
def validate_referential_integrity(parent_ids: list, child_ids: list, relationship: str) -> dict:
    """Valida integridad referencial con explicación."""
    orphans = set(child_ids) - set(parent_ids)
    
    prompt = f'''
Relación: {relationship}
IDs huérfanos (en tabla hija pero no en padre): {list(orphans)[:10]}
Total huérfanos: {len(orphans)}

Explica el problema y sugiere 3 posibles causas.
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.2
    )
    
    return {
        'orphans_count': len(orphans),
        'orphan_sample': list(orphans)[:5],
        'explanation': resp.choices[0].message.content
    }

# Ejemplo
productos_ids = [1, 2, 3, 4, 5]
ventas_producto_ids = [1, 2, 3, 99, 100, 5]

integrity_check = validate_referential_integrity(
    parent_ids=productos_ids,
    child_ids=ventas_producto_ids,
    relationship='ventas.producto_id -> productos.producto_id'
)

print(f"Huérfanos: {integrity_check['orphans_count']}")
print(f"Muestra: {integrity_check['orphan_sample']}")
print(f"\nExplicación:\n{integrity_check['explanation']}")

## 8. Buenas prácticas

- **Complementar, no reemplazar**: LLMs complementan herramientas tradicionales (GE, pandas profiling).
- **Validación humana**: revisa sugerencias antes de aplicar.
- **Umbrales**: define confidence thresholds para automatización.
- **Logging**: registra todas las decisiones del LLM.
- **Costos**: cachea validaciones comunes.
- **Testing**: valida el validador con datos conocidos.

## 9. Ejercicios

1. Construye un sistema de auto-reparación de datos usando LLM suggestions.
2. Genera un data quality dashboard con explicaciones en lenguaje natural.
3. Implementa detección de PII (datos sensibles) con LLMs.
4. Crea un agente que diagnostique problemas de data quality y proponga fixes.